In [1]:
import pandas as pd

from hardware.opentrons.opentrons_api import OpentronsAPI

from hardware.cameras import PendantDropCamera

from hardware.sensor.sensor_api import SensorAPI
from hardware.opentrons.formulater import Formulater

from hardware.opentrons.configuration import Configuration
from analysis.plots import Plotter

from utils.logger import Logger
from utils.load_save_functions import load_settings
from utils.search_containers import get_well_id_concentration, get_well_id_solution
from analysis.active_learning import ActiveLearner
from analysis.models import szyszkowski_model

from hardware.opentrons.droplet_manager import DropletManager
from utils.load_save_functions import initialize_results


opentrons_api = OpentronsAPI()

opentrons_api.initialise()

config = Configuration(opentrons_api=opentrons_api)

labware = config.load_labware()

containers = config.load_containers()

pipettes = config.load_pipettes()

left_pipette = pipettes["left"]

right_pipette = pipettes["right"]

pendant_drop_camera = PendantDropCamera()
sensor_api = SensorAPI()
plotter = Plotter()

formulater = Formulater(
    left_pipette=left_pipette,
    right_pipette=right_pipette,
    containers=containers,
)

droplet_manager = DropletManager(
    left_pipette=left_pipette,
    containers=containers,
    pendant_drop_camera=pendant_drop_camera,
    opentrons_api=opentrons_api,
    plotter=plotter,
)

c:\Users\pimda\anaconda3\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-03-26 09:41:24,913 - INFO - Protocol already uploaded, using existing protocol.
2025-03-26 09:41:32,327 - INFO - Run created succesfully (ID: a09135a2-e0a8-4b64-8afd-d732a864e328).
2025-03-26 09:41:35,009 - INFO - All custom labware definitions added.
2025-03-26 09:41:36,307 - INFO - Labware loaded successfully.
2025-03-26 09:41:36,382 - INFO - Containers loaded successfully.
2025-03-26 09:41:40,552 - INFO - Pipettes loaded successfully.


In [10]:
opentrons_api.home()

2025-03-26 09:44:18,690 - INFO - Robot homed.


In [3]:
well_id = "7H12"

## Focus camera

In [11]:
left_pipette.pick_up_needle()

2025-03-26 09:45:46,640 - INFO - Picked up needle.


In [12]:
left_pipette.aspirate(
    volume=17,
    source=containers[well_id]
)
left_pipette.air_gap(air_volume=3)
left_pipette.clean_on_sponge()
left_pipette.remove_air_gap(at_drop_stage=True)

2025-03-26 09:46:01,723 - INFO - Container: aspirated 17 uL from this container with content pure mM empty.
2025-03-26 09:46:01,723 - INFO - Aspirated 17 uL from 7H12 with left pipette.
2025-03-26 09:46:05,660 - INFO - Air gap of 3 uL performed in left pipette.
2025-03-26 09:46:10,139 - INFO - Tip/needle cleaned on sponge.
2025-03-26 09:46:15,308 - INFO - Air gap of 3 uL removed in left pipette.


In [15]:
left_pipette.dispense(
    volume=5, destination=containers["drop_stage"], flow_rate=1, depth_offset=-23.4
)

2025-03-26 09:46:54,289 - INFO - Dispensed 5 uL into well 10A1 with left pipette.


In [16]:
left_pipette.aspirate(volume=11, source=containers['drop_stage'], depth_offset=-23.4)

2025-03-26 09:49:32,457 - INFO - Aspirated 11 uL from 10A1 with left pipette.


In [17]:
left_pipette.dispense(volume=17, destination=containers[well_id])

2025-03-26 09:49:39,240 - INFO - Container: dispensed 17 uL into this container from source 10A1 containing pure mM empty.
2025-03-26 09:49:39,240 - INFO - Dispensed 17 uL into well 7H12 with left pipette.


In [ ]:
# formulater.wash(repeat=3)

2025-03-25 14:31:07,398 - INFO - Right pipette dropped tip into trash.
2025-03-25 14:31:15,178 - INFO - Picked up tip.
2025-03-25 14:31:25,040 - INFO - Touched tip performed, repeated 1 times
2025-03-25 14:31:25,056 - INFO - Container: aspirated 300 uL from this container with content pure mM water.
2025-03-25 14:31:25,056 - INFO - Aspirated 300 uL from 2A1 with right pipette.
2025-03-25 14:31:32,806 - INFO - Touched tip performed, repeated 1 times
2025-03-25 14:31:32,806 - INFO - Container: dispensed 300 uL into this container from source 2A1 containing pure mM water.
2025-03-25 14:31:32,806 - INFO - Dispensed 300 uL into well 3A1 with right pipette.
2025-03-25 14:31:50,158 - INFO - Done with mixing in 3A1 with order after, with volume 20 uL, repeated 5 times
2025-03-25 14:32:00,523 - INFO - Touched tip performed, repeated 1 times
2025-03-25 14:32:00,539 - INFO - Container: aspirated 300 uL from this container with content pure mM water.
2025-03-25 14:32:00,540 - INFO - Aspirated 300 

In [18]:
left_pipette.return_needle()

2025-03-26 09:49:56,640 - INFO - Returned needle.


## Get scale

In [ ]:
drop_parameters = {
    "drop_volume": 11,
    "max_measure_time": 60,
    "flow_rate": 1,
}
well_id = "7A12"

In [40]:
scale_t, drop_parameters = droplet_manager.measure_pendant_drop(source=containers[well_id], drop_parameters=drop_parameters, calibrate=True)

2025-03-25 14:34:15,798 - INFO - Start measurment of pendant drop of 7A12 with drop volume 11.0 uL and drop count 1.
2025-03-25 14:34:24,925 - INFO - Done with mixing in 7A12 with order before, with volume 15 uL, repeated 3 times
2025-03-25 14:34:26,496 - WARNING - Aspiration volume is larger than container volume!
2025-03-25 14:34:26,497 - INFO - Aspirated 17 uL from 7A12 with left pipette.
2025-03-25 14:34:30,406 - INFO - Air gap of 3 uL performed in left pipette.
2025-03-25 14:34:34,679 - INFO - Tip/needle cleaned on sponge.
2025-03-25 14:34:39,895 - INFO - Air gap of 3 uL removed in left pipette.
2025-03-25 14:34:39,895 - INFO - Dispensing pendant drop.
2025-03-25 14:34:51,917 - INFO - Camera: start measuring 7A12.
2025-03-25 14:35:53,442 - INFO - Camera: stopped measurement
2025-03-25 14:35:54,397 - INFO - Re-aspirated the pendant drop into the tip.
2025-03-25 14:35:58,112 - INFO - Container: dispensed 17 uL into this container from source 7A12 containing n.a. mM empty.
2025-03-25

In [1]:
left_pipette.return_needle()

NameError: name 'left_pipette' is not defined

In [41]:
from utils.utils import calculate_average_in_column

print(f"average scale: {calculate_average_in_column(x=scale_t, column_index=1)}")

average scale: 0.002651306569856702
